In [ ]:
!pip install conllu

In [ ]:

import conllu
from io import open

file_path = '/content/processed_all.conllu.2.2.2.modified'

with open(file_path, "r", encoding="utf-8") as f:
    data = f.read()

sentences = conllu.parse(data)

print(f"\nSuccessfully read {len(sentences)} sentences from '{file_path}'.")

if sentences:
    print(f"The first sentence has {len(sentences[0])} tokens.")
    print("First token of the first sentence:")
    print(sentences[0][0])


Attempting to read: /content/processed_all.conllu.2.2.2.modified
Ensure this file is uploaded to your Colab session's root directory.

Successfully read 17897 sentences from '/content/processed_all.conllu.2.2.2.modified'.
The first sentence has 7 tokens.
First token of the first sentence:
В


In [ ]:
modified_tokens_count = 0
modified_sentences_indices = set()

for sentence_idx, sentence in enumerate(sentences):
    for token_index, token in enumerate(sentence):
        if token['deprel'] == 'nsubj':
            head_id = token['head']

            if head_id is not None and head_id > 0:
                if head_id <= len(sentence):
                    head_token = sentence[head_id - 1]
                    if head_token['id'] == head_id:
                        if head_token['upos'] == 'VERB':
                            if head_token['feats'] and head_token['feats'].get('Voice') == 'Pass':
                                token['deprel'] = 'nsubj:pass'
                                modified_tokens_count += 1
                                modified_sentences_indices.add(sentence_idx)

print(f"\nProcessing complete.")
print(f"Number of tokens modified: {modified_tokens_count}")
print(f"Number of sentences affected: {len(modified_sentences_indices)}")

if modified_tokens_count > 0:
    print("\nExample of a modified sentence (if any were modified):")
    if modified_sentences_indices:
        example_idx = list(modified_sentences_indices)[0]
        print(f"--- Displaying Sentence Index {example_idx} (sent_id: {sentences[example_idx].metadata.get('sent_id', 'N/A')}) ---")
        print(sentences[example_idx].serialize())

    modified_conllu_text = "".join([s.serialize() for s in sentences])

    print(f"\nFirst 1500 characters of the modified CoNLL-U output:\n")
    print(modified_conllu_text[:1500])

    output_file_path = 'processed_all.conllu.2.2.2.modified'
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(modified_conllu_text)
    print(f"\nModified CoNLL-U data saved to: {output_file_path}")
else:
    print("No tokens met all conditions for modification.")

Processing 17897 sentences...

Processing complete.
Number of tokens modified: 0
Number of sentences affected: 0
No tokens met all conditions for modification.


In [ ]:
import json
from collections import OrderedDict
from io import open as io_open

comments_json_path = 'comments.json'
posts_json_path = 'posts.json'
output_conllu_enriched_formatted_path = 'processed_all.conllu.2.2.2.meta_enriched_formatted.conllu'

source_pool = []
json_data_loaded = False
try:
    with io_open(comments_json_path, 'r', encoding='utf-8') as f:
        comment_data = json.load(f)
        for item in comment_data: item['type'] = 'comment'; source_pool.append(item)
    print(f"Loaded {len(comment_data)} items from '{comments_json_path}'.")
    with io_open(posts_json_path, 'r', encoding='utf-8') as f:
        post_data = json.load(f)
        for item in post_data: item['type'] = 'post'; source_pool.append(item)
    print(f"Loaded {len(post_data)} items from '{posts_json_path}'.")
    print(f"Total items in source_pool: {len(source_pool)}")
    json_data_loaded = True
except FileNotFoundError as e: print(f"\nERROR: JSON file not found: {e.filename}")
except json.JSONDecodeError as e: print(f"\nERROR: Could not decode JSON: {e}")
except Exception as e: print(f"\nAn unexpected error occurred while loading JSON data: {e}")

if json_data_loaded and ('sentences' in locals() and sentences):
    last_used_source_pool_idx = -1
    print(f"\nProcessing {len(sentences)} CoNLL-U sentences for metadata enrichment and formatting...")

    for conllu_sentence_obj in sentences:
        original_meta = conllu_sentence_obj.metadata
        conllu_text_for_matching = original_meta.get('text', '').strip()
        is_newpar = 'newpar' in original_meta

        new_meta_ordered = OrderedDict()

        if is_newpar:
            new_meta_ordered['newpar'] = None

        new_meta_ordered['sent_id'] = original_meta.get('sent_id')

        found_source_item = None
        if is_newpar:
            for i in range(last_used_source_pool_idx + 1, len(source_pool)):
                candidate_item = source_pool[i]
                candidate_text = candidate_item.get('text', '').strip()
                if candidate_text.startswith(conllu_text_for_matching):
                    found_source_item = candidate_item
                    last_used_source_pool_idx = i
                    break

            if found_source_item:
                formatted_source = None
                formatted_speaker = None
                date_val = str(found_source_item.get('date', 'DATE_UNKNOWN'))

                if found_source_item['type'] == 'post':
                    post_item_id = found_source_item.get('id')
                    post_from_id = found_source_item.get('from_id')
                    if post_from_id is not None and post_item_id is not None:
                        formatted_source = f"vk.com/wall{post_from_id}_{post_item_id}"
                    if post_from_id is not None:
                        formatted_speaker = f"vk.com/wall{post_from_id}"

                elif found_source_item['type'] == 'comment':
                    comment_owner_id = found_source_item.get('owner_id')
                    comment_post_id = found_source_item.get('post_id')
                    comment_from_id = found_source_item.get('from_id')
                    if comment_owner_id is not None and comment_post_id is not None:
                        formatted_source = f"vk.com/wall{comment_owner_id}_{comment_post_id}"
                    if comment_from_id is not None:
                        formatted_speaker = f"vk.com/id{comment_from_id}"

                if formatted_source:
                    new_meta_ordered['source'] = formatted_source
                if formatted_speaker:
                    new_meta_ordered['speaker'] = formatted_speaker
                new_meta_ordered['date'] = date_val

            elif conllu_text_for_matching:
                print(f"Warning: No matching JSON source for #newpar sentence (sent_id: {original_meta.get('sent_id')}) starting: '{conllu_text_for_matching[:50]}...'")

        new_meta_ordered['genre'] = 'social'

        new_meta_ordered['text'] = original_meta.get('text')

        for key, value in original_meta.items():
            if key not in new_meta_ordered:
                new_meta_ordered[key] = value

        conllu_sentence_obj.metadata = new_meta_ordered

    final_enriched_formatted_conllu_string = "".join([s.serialize() for s in sentences])

    try:
        with io_open(output_conllu_enriched_formatted_path, "w", encoding="utf-8") as f:
            f.write(final_enriched_formatted_conllu_string)
        print(f"\nSuccessfully processed and saved enriched CoNLL-U data to: {output_conllu_enriched_formatted_path}")
        print(f"\nFirst 1500 characters of the new file '{output_conllu_enriched_formatted_path}':")
        print(final_enriched_formatted_conllu_string[:1500])
    except Exception as e:
        print(f"\nError writing output file '{output_conllu_enriched_formatted_path}': {e}")

elif not json_data_loaded:
    print("\nSkipping metadata enrichment in Cell 2 because JSON data could not be loaded.")
elif not ('sentences' in locals() and sentences and not isinstance(sentences[0], type(DummySentence))):
    print("\nSkipping metadata enrichment in Cell 2 because 'sentences' (CoNLL-U data from Cell 1) is not available or empty.")


Starting metadata enrichment process with URL formatting...
Reading JSON data from 'comments.json' and 'posts.json'.
Loaded 8474 items from 'comments.json'.
Loaded 1000 items from 'posts.json'.
Total items in source_pool: 9474

Processing 17897 CoNLL-U sentences for metadata enrichment and formatting...

Successfully processed and saved enriched CoNLL-U data to: processed_all.conllu.2.2.2.meta_enriched_formatted.conllu

First 1500 characters of the new file 'processed_all.conllu.2.2.2.meta_enriched_formatted.conllu':
# newpar
# sent_id = 1
# source = vk.com/wall-37725105_5082188
# speaker = vk.com/id330213862
# date = 1743681381
# genre = social
# text = В закладочку на страницу пожалуй добавлю)
# newdoc
# generator = UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe
# udpipe_model = russian-taiga-ud-2.15-241121
# udpipe_model_licence = CC BY-NC-SA
1	В	в	ADP	_	_	2	case	_	TokenRange=0:1
2	закладочку	закладочка	NOUN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	6	obl	_	TokenRange=2:

In [ ]:
failed_ids = '''
Warning: No matching JSON source for #newpar sentence (sent_id: 45) starting: 'Я хочу спросить может какой-то троллейбус пустят д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 74) starting: 'У нас такая погода Сегодня солнечно а завтра опять...'
Warning: No matching JSON source for #newpar sentence (sent_id: 92) starting: 'Нечему ошибки других не учат То я помню у пенсионе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 156) starting: 'Пусть лучше скоростью нормальную сделают Роскомнад...'
Warning: No matching JSON source for #newpar sentence (sent_id: 278) starting: '⠀ этой весной стало лучше, чем тогда....'
Warning: No matching JSON source for #newpar sentence (sent_id: 310) starting: 'Всем привет🖐, прогулка в лесу после работы и чаёк ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 352) starting: 'Если и купаться, то с 8 до 11,12 , потом - мутняк😜...'
Warning: No matching JSON source for #newpar sentence (sent_id: 409) starting: 'Батареи дома перекрыли,но платить всё равно придёт...'
Warning: No matching JSON source for #newpar sentence (sent_id: 412) starting: 'На кой х€р улицу топить?????...'
Warning: No matching JSON source for #newpar sentence (sent_id: 423) starting: 'И естественно находятся гении, которые кулакова пе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 545) starting: 'Когда у тебя первый рабочий день на этой неделе в ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 558) starting: 'Кто-то любит дождь ⛈️, а я солнцезависимая, жду☀️...'
Warning: No matching JSON source for #newpar sentence (sent_id: 564) starting: 'После дождя По скользкому асфальту проходя, По пло...'
Warning: No matching JSON source for #newpar sentence (sent_id: 575) starting: 'Бестолочи(( водитель мерена красавчик, только жаль...'
Warning: No matching JSON source for #newpar sentence (sent_id: 668) starting: 'Нам периодически приходится летать с детьми....'
Warning: No matching JSON source for #newpar sentence (sent_id: 705) starting: 'Это чудо поставили в Анапе, не красиво и не удобно...'
Warning: No matching JSON source for #newpar sentence (sent_id: 718) starting: 'И обесточить GPS модуль ума не хватило😁....'
Warning: No matching JSON source for #newpar sentence (sent_id: 863) starting: 'В Таиланде за написание на купюре в тюрьме бы оказ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 900) starting: '[id759871039|Владимир], Не придумывайте ерунду об ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 915) starting: 'Так не чего не понятно снили или нет...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1046) starting: 'Работаю Это просто интерьер)...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1093) starting: 'Час назад там было все нормально Щас проходил уже ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1127) starting: 'По стране дроны летают, странно вообще что такие м...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1153) starting: 'Эльбрус виден утром из Ставрополя, 3я больница...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1283) starting: 'Тело не хочет делать упражнения сегодня , пожалуй ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1407) starting: 'А 117 штрафов хоть уплаченны?...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1427) starting: 'БМВ+117 штрафов закономерно...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1440) starting: '118-ый штраф выписали?...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1463) starting: 'Дом 🏠 милый дом...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1581) starting: '11 апреля 2025 года отмечается ВСЕМИРНЫЙ ФЕСТИВАЛЬ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1620) starting: 'А почему рядом , говорили, и в газете писали, дом ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1645) starting: 'А что ж.инкасаторы не изъяли эти деньги..или была ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1740) starting: 'Мой парень сказал «правильно», а я от себя добавлю...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1860) starting: 'Регулярная тренировка пресса каждый день способств...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1910) starting: 'Не справедливые подожгли, потому, что никто свой д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 1977) starting: 'Вот логика ограбить в центре города....'
Warning: No matching JSON source for #newpar sentence (sent_id: 2177) starting: 'Сезон открывает Весну призывает...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2204) starting: 'С весной все понятно, давай лето 🤣🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2220) starting: 'Зима сегодня вернулась специально для тех кто не у...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2222) starting: 'Все в мире сейчас загадочно, Все будто летит куда-...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2249) starting: 'После укуса клеща врачи его снимают с тела и говор...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2276) starting: 'Если ползущего раздавить, то он размазывается, а н...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2285) starting: 'Кто-то наварится великолепно...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2296) starting: 'Демонтажные работы: Работа экскаватора, вывоз стар...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2309) starting: 'Пи....ц: аликулиев, еганян, николаевич -это подряд...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2316) starting: '19 мультов за 500 метров, Лучше бы не писали такие...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2319) starting: 'Неистовая гордость за город овладевает от присутст...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2339) starting: 'кстати, эмблема откуда, когда появилась в России?...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2350) starting: 'Вам всем делать нечего? ...... завидно...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2357) starting: 'Я не знаю как у вас пчелки опыляют, у нас просто в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2411) starting: 'Спасибо теперь смс приходят вовремя , перешел на т...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2418) starting: 'РНСтаврополье 09.04.2025 примерно в 05:50 на расст...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2566) starting: 'Чай пьем 😍 И снег .....'
Warning: No matching JSON source for #newpar sentence (sent_id: 2611) starting: 'Мысленно представляю себе аэрохоккей, игру в одног...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2669) starting: 'За стрельбу в этом доме ещё с обеда писали в пабли...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2731) starting: 'Нешуточное себе дело -5....'
Warning: No matching JSON source for #newpar sentence (sent_id: 2855) starting: 'кино смотрю Привет Так же, довожу до вашего сведен...'
Warning: No matching JSON source for #newpar sentence (sent_id: 2951) starting: 'С балкона по нему с воздушки самое то!...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3009) starting: 'Человек ни кого не трогает живёт так как хочет ваш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3010) starting: 'у знакомого клубмбу как кабаны в лесу всю перерыли...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3195) starting: 'Сегодня был вкусный ужин , повезло -пожарили , усп...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3202) starting: 'День 4 из 30 планка+ пресс ✅Мой плейлист ...что - ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3212) starting: 'Сегодня зачищали и грунтовали стены....'
Warning: No matching JSON source for #newpar sentence (sent_id: 3227) starting: 'по контракту работы по разгону града начинаются то...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3268) starting: 'За$%##%%о долетались со своими ионами йодистого се...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3269) starting: 'Мы предупреждали, что будет градовый шторм в выдел...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3274) starting: 'а работы по разгону градовых тучь по контракту нач...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3307) starting: 'В подьезде живёт просто больной человек, хотя зани...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3369) starting: 'Расчёт картой,водители часто говорят,оплата не про...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3423) starting: 'Мне одному кажется кривой формулировка: УМЫШЛЕННОЕ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3464) starting: '❤На одном из форумов 25 летняя Ипатовская задала в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3607) starting: 'Это ОПГ ☝ И преднамеренное убийство....'
Warning: No matching JSON source for #newpar sentence (sent_id: 3634) starting: 'Все темные пятна в черте города - это ниши леса ((...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3683) starting: 'ставропольЧАНЕ пятигорЧАНЕ кисловодЧАНЕ не кислово...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3766) starting: 'Я с Ставрополье как же всех за ну понимаете о чем ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3811) starting: 'Нация бесполезная для государства, пособия получаю...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3886) starting: 'Как по мне -так все верно, во первых зачем человек...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3954) starting: 'Кажется, на агрессивность тестировать пора не толь...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3979) starting: 'Внезапно : ❗Информация о нападении подростков на д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 3986) starting: 'Что за чешую несут , не могут найти , камер по нав...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4004) starting: 'Так почти все специалисты работают в платных клини...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4015) starting: 'Дык и сейчас некоторые функции лечащего врача оказ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4016) starting: 'Зато в мед не поступить.Верните назад интернатуру,...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4135) starting: 'Прилетела девушка в Крым, а её багаж улетел в друг...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4215) starting: 'Тут и дебилу понятно што убили намеренно, и списат...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4263) starting: 'Дразнят псов, ни разу не видел, чтобы Набрасывалис...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4269) starting: 'Бредовый пост ни о чем, обычный высер высосанный и...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4283) starting: 'Да уберите вы этих собак с улиц, не должны они бег...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4333) starting: 'Отдайте этой мамаше флаг пусть выстирает и поглади...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4425) starting: '[id843169753|Вадим], Да Вадим единственный и первы...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4492) starting: 'Я могу одно сказать ., ели любит то родит одного р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4517) starting: 'Не знаю ,кто это , но каждый должен нести наказани...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4563) starting: 'Крышу сорвало походу пора им в гости в психушку ....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4596) starting: 'Дитя пьяного зачатия алкоголиков...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4614) starting: 'Блин я когда так убегал , меня трое с 2 экипажей д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4632) starting: 'Господи, слава богу мы развелись по старым ценам ....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4663) starting: 'С хера ли, веселые твари, дебилизм полный...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4717) starting: 'Добрый день я очевидец прое произошедшего конфликт...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4718) starting: 'Не, ну эт точно поможет, прям верняк 100%, если ср...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4760) starting: 'Люди, я не против танцев....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4767) starting: 'Вот я вижу все эти оскорбительные комментарии....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4836) starting: 'А санэпидем как?Подземные воды рядом Дебил только ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4846) starting: 'Так мы и не знаем цены на лимоны....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4861) starting: 'Да все в этом возрасте и цветы рвали, и петрушку и...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4868) starting: 'Кто-то ухаживает , старается , а кто то считает чт...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4880) starting: 'Скоро мужчин, пожимающим друг другу руки при встре...'
Warning: No matching JSON source for #newpar sentence (sent_id: 4906) starting: 'Весеннее обострение началось или просто у кого то....'
Warning: No matching JSON source for #newpar sentence (sent_id: 4914) starting: 'Коты дуреют с этой прикормки Осторожно!😅...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5011) starting: 'Квартиры посуточно никто не отменял Да и в машине ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5074) starting: 'Памятник вокзалу 😂😂😂...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5088) starting: 'Автовокзал это объект критической инфраструктуры, ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5093) starting: 'Вот смотрите это было 18 мая 2023 на красной машин...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5094) starting: 'А может хозяева этих машин ина сво служат поэтому ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5148) starting: 'Зато по ТВ всё нормально.В селе 15 лет назад выпус...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5166) starting: 'Повышать повышать повышать Может все же вернете вс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5170) starting: 'Когда в начале 90х дерьмократы рвались к власти , ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5189) starting: 'Я не понимаю таких людей Пытаются государство наеб...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5196) starting: 'Не голова, а дом советов......'
Warning: No matching JSON source for #newpar sentence (sent_id: 5211) starting: 'Пошли они со своими предложениями, я в девять вече...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5254) starting: 'Если смотреть как играет наша сборная , то ваще до...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5271) starting: 'Да хорош их в попу целовать, реальный срок дайте, ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5304) starting: 'Комментарии по номеру 9887322?48...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5305) starting: 'На СВО на перевоспитание...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5375) starting: 'Вот так работают Чужими руками, а злоумышленники н...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5428) starting: 'Нынче рогатых оленей по стране ходит поболее...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5434) starting: '152-ФЗ: ну да, ну да, пошёл я нафиг...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5448) starting: 'Даже интересно, почему мой комментарий отправили в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5452) starting: 'Так и пиште- заднеприводный что-то там натворил, н...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5453) starting: 'Во, класс, тут написали правильно - из Ставропольс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5600) starting: 'Как по телевизору показывают госдуру так они сами ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5664) starting: 'Где в Москве а тут как были капейки так и будет...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5706) starting: 'Засуньте эту среднюю Московскую и поедете по дерев...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5751) starting: 'Гайцы давно превратились в гиен Это не люди...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5796) starting: 'Увольть с позором, а остальных проверить на проф. ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5800) starting: 'Да россияне своей стране нахуй лет 20 не нужны как...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5838) starting: 'Потом это кровь так боком с выйдет с самого челове...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5888) starting: 'Хамзаев походу самогонку по ночям гонит....'
Warning: No matching JSON source for #newpar sentence (sent_id: 5897) starting: 'Чтобы у него соль покупали по выходным...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5907) starting: 'Начнут в рабояие дни пить😹...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5908) starting: 'Это проблему с алкашами не решит🤦...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5944) starting: 'А если б к горчице туда еще котлету затолкал, то г...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5964) starting: 'Человек умер... люди, оставайтесь людьми!...'
Warning: No matching JSON source for #newpar sentence (sent_id: 5976) starting: 'Ахуеть, царство небесное, В добрый путь 😔...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6111) starting: '2010 г без гнили Жучков ржавчины цена 225 торг на ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6131) starting: '1.8 мех 420.000р Собственник...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6143) starting: 'Современный тренд как можно больше земли сплошь ук...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6182) starting: 'Просто хочется Мира....'
Warning: No matching JSON source for #newpar sentence (sent_id: 6214) starting: 'За последние годы стало холодно и даже в мае,поэто...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6242) starting: 'А кто же будет штрафовать компании по вывозу мусор...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6318) starting: 'А мы отключили отопление многоквартирном доме ещё ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6319) starting: 'https://vk.com/clip639858156_456244182?c=1 Что-то ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6369) starting: 'Нам тупо врут,это где что он озвучил.Мы И Все Жите...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6399) starting: '"Мне кажется порою, что солдаты, С кровавых не при...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6466) starting: 'Всем привет, подскажите может кто знает Как правил...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6556) starting: 'Погода просто кайф нереальный Наконец-то зима норм...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6562) starting: 'Удивительно то, что все ждут работы коммунальных с...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6565) starting: 'Всем ждунам расчистки снега коммунальными службами...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6640) starting: 'А взрослых ,настоящих мужиков,женщин не было в тра...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6666) starting: 'Уже не знаете с кого соскребать))) сначала начните...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6743) starting: 'Судя по почтовому ящику,у дома два хозяина,но если...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6780) starting: 'Идут годы, естественно разрушается....'
Warning: No matching JSON source for #newpar sentence (sent_id: 6800) starting: 'Или ЖК Солдат )) «Для нуждающихся» 200 000 за квад...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6808) starting: 'Их куча и даже с номерами Их реально очень много...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6833) starting: 'Михайловск стал вторым по численности населения го...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6855) starting: 'За то собак везде,скоро не только фазанам,людей не...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6858) starting: 'Ему 🦊 сказал, что скоро % по ипотеке снизится Ходи...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6898) starting: 'ЗНАТНАЯ ШЛЯПАТЕНЬ ВАШ ЖК...'
Warning: No matching JSON source for #newpar sentence (sent_id: 6938) starting: '3 штучки по 5000 есть....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7049) starting: 'Желательно, чтоб было для реабилитации отдельное о...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7084) starting: 'Честно,если есть возможность можно засыпать чем то...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7207) starting: 'А где он стоит я просто не разу не видела живу зде...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7212) starting: 'До второй больницы не добраться с юга, только пере...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7248) starting: 'До каких пор мы налогоплательщики будем содержать ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7261) starting: 'Ходят на красный , по видео видно что движение маш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7271) starting: 'Теплосеть, ВЫКЛЮЧИТЕ ОТОПЛЕНИЕ!!!!!!!!!!!!!!!!...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7276) starting: 'Вот опять окно, Где опять не спят....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7295) starting: 'В детдом его Там его научат взрослых любит чертено...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7353) starting: 'Есть профессиональные военные, пусть отрабатывают,...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7515) starting: 'Колонны, конечно, в стиле "Смех на лужайке" Осталь...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7535) starting: 'Торговцы нарушают законы РФ: — ФЗ №52 «О животном ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7561) starting: 'Какие все привередливые стали Человек может херачи...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7567) starting: 'Нежная какая, а когда намажутся этими духами с рын...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7576) starting: 'Нативная реклама от Водоканала (мойтесь чаще) 🤣🤣🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7580) starting: 'Походу все вонючие защищают вонючих🤔🤣🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7581) starting: 'я бы ещё не пускала в транспорт курящих, задыхаетш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7625) starting: 'Продам хатсан 125 с оптикой ....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7630) starting: 'Цена 4к Оптика, две пачки пуль, мишени...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7640) starting: 'Саламалейкум Как можно связаться с хозяином?...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7644) starting: 'Напиши в лс Заберу...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7706) starting: 'Адреса отключения: https://news.1777.ru/110953-vre...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7711) starting: 'Круто, ОПГ, и штрафы по 50 тыщ рублей, все нечестн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7730) starting: 'Он ее избил , а ей дали 7 суток...человек в аффект...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7740) starting: 'У нас дома холодно и с отоплением....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7749) starting: 'Населенные пункты и адреса отключений: https://new...'
Warning: No matching JSON source for #newpar sentence (sent_id: 7781) starting: 'Зарплата 20 лет назад была и осталась кроме цен....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7940) starting: 'Прав пожизне и машину у таких лишать ....'
Warning: No matching JSON source for #newpar sentence (sent_id: 7996) starting: 'Если бы, ударили не врача, а к примеру не дай бог ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8025) starting: 'Еще больше новинок на сайте магазина https://mania...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8026) starting: 'Да их таких полно… у нас за гаражами каждую весну ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8062) starting: 'Улицы, на которых будет отсутствовать газ 15 и 17 ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8171) starting: 'Шляпа это всё Если есть ты с мозгами,прошаренный с...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8234) starting: 'доброе утро)) чёт Кот давно не ссыт в ПС😄...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8333) starting: 'все что то делали сегодня😄)) вот и я заморочился))...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8458) starting: 'немного математики: 31+14=45 45- мужик ягодка опят...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8566) starting: 'Я тоже разбирал приёмники и магнитофоны, и не зря,...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8575) starting: '😅 Я так в детстве родительский видак разобрал 🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8600) starting: 'Я думала желание собрать компанию и отпраздновать ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8680) starting: 'Твои бесхребетные проблемы....всем пох&й ... и воо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8697) starting: 'Пригласи её в гости, чайку попейте, да и пусть к в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8714) starting: 'Должна радоваться, ты ноешь, что есть, запасной ва...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8731) starting: 'Да, соберите ему вещи, и вперед во взрослую жизнь....'
Warning: No matching JSON source for #newpar sentence (sent_id: 8735) starting: 'Снять хату им пополам с родителями девочки....'
Warning: No matching JSON source for #newpar sentence (sent_id: 8852) starting: 'Если есть доказательства р том, что вы ему дали де...'
Warning: No matching JSON source for #newpar sentence (sent_id: 8911) starting: 'Настаящая женщена,ни будит ганяца за капейками,а е...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9023) starting: 'Савченко с обеда не видно, а в новостях пишут, что...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9054) starting: 'Я тоже могу себя считать скуфом Потому что я работ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9186) starting: 'ПУСТЬ ЕЩЁ, ЖЕНИТЬСЯ, ДЕМОГРАФИЮ НАДО ПОДНИМАТЬ 👍👍👍...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9291) starting: 'Я хотел купить квартиру на мира прям рядом с автов...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9382) starting: 'Автор а зачем живёшь ведь всё рпвно помрёшь, зачем...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9497) starting: 'Бороздить 🐳 🐳🐳 🐳🐳🐳 🐳🐳🐳🐳 🐳🐳🐳🐳🐳 🐳🐳🐳🐳🐳🐳...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9498) starting: '🐳🐳🐳🐳🐳🐳🐳🐳 🐳🐳🐳🐳🐳🐳🐳🐳🐳 ⚓⚓⚓⚓⚓⚓⚓⚓⚓⚓...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9581) starting: 'кто скажет, зачем я это всё прочитал😄) спок.ночи))...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9660) starting: 'Ты находишься среди тех людей которые тебе не подх...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9838) starting: 'найди кошку😁)) кст. я купил печку с казаном😁) кида...'
Warning: No matching JSON source for #newpar sentence (sent_id: 9887) starting: 'Кок у нас на лодке...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10010) starting: 'А волны и стонут, и плачут, И плещут на борт кораб...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10011) starting: 'Растаял в далеком тумане Рыбачий, Родимая наша зем...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10012) starting: 'Подлодку упрямо качает Крутая морская волна, Подни...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10016) starting: 'Нелегкой походкой матросской Иду я навстречу врага...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10018) starting: 'Хоть волны и стонут и плачут И плещут на борт кора...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10056) starting: '# Работа промоутером: расклейка объявлений в 10 го...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10164) starting: 'И никак не дойду Иду и иду...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10178) starting: 'Нет, это возможность стать богатым в любой момент ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10269) starting: 'Пипец проблема Твой ребёнок берет нож у него поигр...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10319) starting: 'Многие комментаторы не понимают опасности ношения ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10323) starting: 'Мы в школу ходили кто с топором, кто в вилами....'
Warning: No matching JSON source for #newpar sentence (sent_id: 10472) starting: 'Сидит бабушка до поздней ночи, продаёт ромашки, на...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10474) starting: 'Достав из дермантина свою мелочь, Зарытую в платок...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10477) starting: 'Я, бесплатно, даю продукты, если, такая ситуация...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10500) starting: 'ну попроси и ты свои подарки обратно, ну если дари...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10546) starting: 'Эх, ты Интеллект, вот что важно....'
Warning: No matching JSON source for #newpar sentence (sent_id: 10553) starting: 'Согласна,ставлю лайк.Неудавшиеся ни в чем в жизни ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10631) starting: '🐊🐊🐊. 🐊🐊🐊...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10632) starting: '🐊🐊🐊. 🐊🐊🐊. 🐊...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10633) starting: '🐊🐊🐊. 🐊🐊🐊. 🐊🐊...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10733) starting: 'Господи, ну кто такое рассказывает 🤦‍♀️ дурында 🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10789) starting: 'Читая комменты "мужчин" становится ясно почему у д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10861) starting: 'воспитывать надо на своём примере)) или говорить: ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10862) starting: 'Может Вас "перетянуть" раз 10....чтобы, у вас, охо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10882) starting: 'Вот знаете у моей подруги 3 детей....'
Warning: No matching JSON source for #newpar sentence (sent_id: 10971) starting: 'На Перспективном, идёшь по тротуарам, и на газонах...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10981) starting: 'Сон мне снится - вот те на: Гроб среди квартиры, Н...'
Warning: No matching JSON source for #newpar sentence (sent_id: 10986) starting: 'Живых бойся,слабенький какой ,представляю что с то...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11024) starting: 'Жуткие ощущения когда ложишься в гроб и кто то зак...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11283) starting: 'Адальберт; 1 Адольф; 12 Айко; 1 Алан; 1 Алоиз; 1 А...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11327) starting: 'Когда-то сын перешëл в новую школу, я писала родит...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11407) starting: 'НОРМАЛЬНО ВСЁ, НЕ ПАРЬСЯ, ИМ БАБКИ НУЖНЫ, ТАК, ЧТО...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11415) starting: 'У таких нерях самооценка пздц... и у мужиков замус...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11453) starting: 'Берегите свою честь с молодости и не будете одинок...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11454) starting: 'Собственно дело то в соответствии внутреннего внеш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11490) starting: 'сегодня у Савченко праздник)) день гречки будет от...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11498) starting: '🕙 🕚...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11600) starting: '⏰ 🕙 🕚...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11623) starting: 'Влюбиться можно в красоту, но полюбить лишь только...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11635) starting: 'Песню вспомнил)) ..Открою я сейчас один секрет - В...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11643) starting: 'Как сказал когда-то Боб Марли: Женщина не должна б...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11648) starting: 'Все женщины с красивыми мужчинами раньше были крас...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11742) starting: 'вонять? 🤔 не пост, а вброс!...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11745) starting: 'В КАЩЕНКО, ИДИ БЫСТРЕЙ, БАШКУ ЛЕЧИ, АФОНЯ👹👈🤣...'
Warning: No matching JSON source for #newpar sentence (sent_id: 11850) starting: 'Любовь не знает логики, она выше разума....'
Warning: No matching JSON source for #newpar sentence (sent_id: 11889) starting: 'Пистос, 20 лет в браке, это же как ты нереалистичн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12163) starting: 'https://vk.com/wall731724434_373 ⏰...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12165) starting: '🕙 🕚 🌞...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12181) starting: 'Это для них возможность сделать шабашку....'
Warning: No matching JSON source for #newpar sentence (sent_id: 12238) starting: 'У вас тут бот завёлся, почти в каждом посте...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12357) starting: 'А когда он предлагал заказать стейк, нужно было ут...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12383) starting: 'Кто приглашает тот и платит, ну и тарелочниц тоже ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12384) starting: 'Кто приглашает тот и платит, ну и тарелочниц тоже ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12417) starting: 'Нужно сразу домой приглашать 😄...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12426) starting: 'Не встречала таких никогда, хотя жила в разных гор...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12449) starting: 'А, верить надо Точнее доверять, не всё говно вокру...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12451) starting: 'МУЖИЧКИ, СЕЙЧАС ТАКИЕ, ЖАДНЫЕ , КАК ЧЕРТИ, БУДТЕ А...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12479) starting: 'почему он то глупый 😐не ты так другая, вас много о...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12497) starting: 'Сводил в ресторан при первой встрече , на второй п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12535) starting: 'Новые троллейбусы начнут работу в Ставрополе с 1 и...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12558) starting: '❗️Жительница Ставрополя перевела мошенникам более ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12580) starting: 'В Ставрополь уже доставили 23 новых троллейбуса с ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12603) starting: 'Еще на восьми маршрутах Ставрополя нашли нарушения...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12617) starting: 'В Ставрополе пока не будут отключать отопление В б...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12632) starting: 'Из-за дорожного ремонта на подъезде к Ставрополю о...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12635) starting: '❗️В Ставрополе заключен под стражу подозреваемый в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12644) starting: 'Сквер планируют обустроить возле аэропорта Ставроп...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12653) starting: 'Житель КЧР украл в Ставрополе прокатный самокат 48...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12673) starting: '❗️На Ставрополье разработали правила для работы шк...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12689) starting: 'Гастрономический фестиваль пройдет 31 мая в Ставро...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12699) starting: 'В Ставрополе проживает более 560 тысяч человек Сог...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12708) starting: 'Водитель со 117 штрафами сбил ребёнка на «зебре» в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12746) starting: '❗️Полицейские задержали двоих человек, пытавшихся ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12753) starting: 'Жительница Ставрополя задушила своего сожителя пос...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12773) starting: 'Два человека пострадали в пожаре на северо-западе ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12798) starting: '❗️Неизвестный взорвал банкомат в Ставрополе Происш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12805) starting: '20 человек, включая 3 детей, эвакуировали из-за по...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12810) starting: 'Сегодня в Ставрополе и еще двух округах проходит т...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12821) starting: 'В Ставропольском крае зафиксировано 154 случая уку...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12825) starting: 'Почти 19 млн рублей выделили на ремонт подпорной с...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12865) starting: 'Полиция выясняет обстоятельства стрельбы в одной и...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12886) starting: 'В Ставрополе начались работы по созданию дополните...'
Warning: No matching JSON source for #newpar sentence (sent_id: 12906) starting: 'Градобойные установки заработают на Ставрополье 1 ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13073) starting: 'Магнолия в Ессентуках 🌸 📸 [club222485942|ЮЖНЫЕ СНЫ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13076) starting: 'Центр Ставрополя уже готовят к «Школьной весне» 🎓 ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13157) starting: 'Волшебная ночь в Кисловодске 🌲 📸 [club222485942|ЮЖ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13235) starting: 'Закат на Орлиных скалах 🌄 📸...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13243) starting: 'Каким бы был Михаил Юрьевич Лермонтов, окажись он ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13300) starting: 'Наши слоняры! 🐘...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13425) starting: '❗Информация о нападении подростков на девочку в Ки...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13431) starting: 'Не раздвигать ноги до свадьбы потребовал от россия...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13443) starting: '❗️Отбирать дипломы у тех, кто не работает по специ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13445) starting: 'Подростки зверски избили скакалкой 8-летнюю девочк...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13449) starting: '❗️Россиян будет лечить средний медперсонал Минздра...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13453) starting: 'В 11 школе Георгиевска группа подростков издевалас...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13461) starting: 'Письмо в редакцию В это воскресенье уже Пасха, и в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13480) starting: 'Летающие чемоданы в аэропорту Минеральных Вод прив...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13487) starting: 'Похитили, били, вымогали миллион: трёх рэкетиров з...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13496) starting: 'Срывавшая никаб в московском метро женщина оказала...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13508) starting: '🙈 Автомобилист развернулся и пробуксовал по чужой ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13513) starting: '«Моего сына пытали и поливали кипятком»: мама звер...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13523) starting: 'Неизвестный пешеход погиб под колёсами иномарки в ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13536) starting: 'На агрессивных собак снова жалуются жители Ставроп...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13540) starting: 'Крупную наркоплантацию выявили в Изобильненском ок...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13548) starting: 'В Госдуме попросили усилить контроль за ценами на ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13558) starting: 'Девушка выставила ряд условий своему мужчине, посл...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13565) starting: 'Блогера в Ставрополе заподозрили в приставании к 1...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13577) starting: 'Более 700 тысяч рублей потратят на отдых ставропол...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13587) starting: 'Российские школьники массово забиваются тату дешёв...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13591) starting: '🏃‍♂ Пьяный марафонец попытался скрыться бегством о...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13604) starting: 'Спасший 12 сослуживцев уроженец Ставрополя получил...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13621) starting: 'Сократить женщинам рабочий день на час предложили ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13626) starting: 'Водитель-новичок отправил на тот свет пешехода в И...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13631) starting: 'Россиянам придётся платить больше: на этот раз за…...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13653) starting: 'Прохожие застали двух котов за серьёзной разборкой...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13654) starting: '❗️Запретить шоу «Беременна в 16» хотят в России В ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13675) starting: 'Гитлер не убивал себя в 1945 году — ЦРУ рассекрети...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13678) starting: 'Злостный нарушитель сбил 8-летнюю девочку в Ставро...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13683) starting: 'Подростки угнали машину и устроили смертельную ава...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13690) starting: '64 года назад Юрий Гагарин открыл для человечества...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13695) starting: 'Рождаемость в России упала до уровня 18 века За пе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13699) starting: 'Не самое удачное ограбление в Ставрополе Руководст...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13706) starting: 'Ипотечные выходные в честь Дня Космонавтики в «ЮгС...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13712) starting: 'Россияне могут получить право отказаться от водите...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13714) starting: 'Жил в землянке и совершал кражи: на Ставрополье по...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13751) starting: 'Пятигорчанин вынес деньги из букмекерской конторы,...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13760) starting: 'Вечный огонь в Невинномысске в очередной раз оскве...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13795) starting: 'Вандалы, осквернившие памятник «Вечная Слава» в Не...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13801) starting: 'Машину с 580 кг взрывчатки, ехавшую в Пятигорск, з...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13816) starting: 'Беспилотник над Ставропольем сбили утром 9 апреля ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13821) starting: 'Создание реестра женатых россиян одобрили в Госдум...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13833) starting: '🔥Из-за удара молнии произошел пожар на ферме Вблиз...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13838) starting: 'АвтоВАЗ приостановит производство Lada Granta - СМ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13840) starting: 'Два человека пострадали в ДТП возле СКФУ Вечером 7...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13844) starting: 'Продукты из насекомых появятся в России в ближайши...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13846) starting: 'Парень напал с палкой на компанию людей в Пятигорс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13860) starting: '43-летнюю наркозакладчицу с крупной партией метилэ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13868) starting: 'До 140 тысяч рублей взлетят зарплаты россиян уже в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13880) starting: 'В Сибири задержали участницу экстремистского сообщ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13898) starting: '😁 Обнаружен эффективный способ избавления от глист...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13916) starting: 'Скандально известный рэппер Паша Техник умер в бол...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13918) starting: 'Пенсионерка "инвестировала" в лже-биржу 7,5 миллио...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13925) starting: 'Фура влетела в отбойник за бывшим Левокумским пост...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13927) starting: 'В Ставрополе подростка на самокате сбили на пешехо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13932) starting: 'В Шпаковском округе возбуждено уголовное дело по ф...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13948) starting: '+1 к фобиям Сегодня на Южном обходе Ставрополя гру...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13950) starting: 'Ночью на проспекте Кулакова автомобиль врезался в ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 13954) starting: 'В Буденновске бдительные граждане помогли задержат...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14021) starting: 'Лада приора 2011 год Пробег 250тыс Цена 380тыс 898...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14031) starting: 'Продам Ваз 2107,2004 год....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14040) starting: '8 год Полный люкс, все работает Двигатель после ре...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14042) starting: 'Продам ваз 2114 в хорошем состоянии цвет перламутр...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14043) starting: 'Bentley Continental GT - 350.000₽ Ford Focus 1 - 4...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14046) starting: 'Приора 2013 г состояние нормальное требует мелких ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14091) starting: 'Обменяю или продам Toyota Curren 280л.с Цена 700т....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14117) starting: 'Продаю: ✅ Infinity QX70 "S"- Дизель 238л.с. ✅...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14168) starting: 'Акцент 2008г Гур,кондер,лобовое целое....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14171) starting: 'Ваз 2112 2005 год Продам автомобиль в хорошем сост...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14198) starting: 'Продам обмен год 2008 на полном ходу цена 850т...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14199) starting: 'Продам обмен Infiniti FX35 3.5 AT год 2008 на полн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14200) starting: '1.6 на полном ходу любые проверки Документы криста...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14226) starting: 'ВАЗ 2112 2003 г.в 8кл цвет кориандр по кузову хоро...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14227) starting: 'Продается ваз 2110 2001год. авто в нормальном сост...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14252) starting: 'Краснодар Мерседес МЛ 500 - двигатель м113 в паре ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14266) starting: 'Продам гранту Конец 19 года Мотор масло не ест Кор...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14282) starting: 'Продам приору в красивом цвете,2009года выпуска Ко...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14291) starting: 'BMW X1 XDRIVE28I 2020г.в....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14327) starting: 'Продам 2105 в хорошем состоянии....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14454) starting: 'Звоните , пишите , машина в хорошем состояние, за ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14479) starting: '99-ка 2000 год Доки чистые штрафов арестов нету Ма...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14503) starting: 'OPEL Astra H GTS Машина в бодром состоянии...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14513) starting: 'Продам Passat b5 plus Рест 2 хозяина 1.8 двс Турбо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14531) starting: 'Продаю Митсубиши Аутлендер 2011г.в, в отличном сос...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14540) starting: '150 000р или обмен на 2114, 2112 2008год, 226т про...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14570) starting: '2005 год 124 мотор бодрый масло не ест....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14577) starting: 'В продаже ваз 2109 1993 гв ГБО с отметкой в птс....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14581) starting: 'Краснодар 3.800.000 89180432970...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14582) starting: 'Прoдам xендай 130 2010 г в. 1.6 MT С автосaлонaми ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14591) starting: 'ВАЗ 2107 2008г красилась и варилась в конце 2024 ,...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14624) starting: 'Президент России Владимир Путин провел заседание П...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14657) starting: 'Проект «Формирование комфортной городской среды» п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14674) starting: 'На Ставрополье продолжается модернизация системы т...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14679) starting: '1500 пряников испекут к Пасхе для участников спецо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14687) starting: 'С приходом весны закаты наполняются новыми краскам...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14696) starting: 'Служба по контракту – путь настоящего мужчины Наша...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14703) starting: 'В преддверии 80-летия Победы управление ЗАГС по Ст...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14728) starting: 'Ставропольские бойцы, выполняющие задачи инженерно...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14734) starting: 'На Ставрополье по поручению главы региона продолжа...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14743) starting: 'В историческом парке «Россия – Моя история» в Став...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14811) starting: 'В Ставропольском крае ожидается резкое похолодание...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14814) starting: 'Александр Овечкин — лучший снайпер в истории НХЛ Р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14838) starting: 'Министр ЖКХ края Александр Рябикин высоко оценил з...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14873) starting: 'Глава Ставрополья Владимир Владимиров рассказал о ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14880) starting: 'Жители Благодарненского округа регулярно отправляю...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14883) starting: 'Специалисты Краевого кадрового центра посетили гос...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14902) starting: 'Решительность и хладнокровие — залог победы Старши...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14919) starting: 'На территории храма Святой Троицы села Краснокумск...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14931) starting: 'В Головинском сквере Георгиевска открыли памятник ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14958) starting: 'Почти 240 проектов реализовали по программе местны...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14963) starting: 'Незаметно к концу подходит первый месяц весны Пого...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14967) starting: 'России нужны настоящие герои Стать им может любой....'
Warning: No matching JSON source for #newpar sentence (sent_id: 14971) starting: '[https://vk.com/wall504722293_131281|Завершилось г...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14978) starting: 'Во всех территориях края стартовали муниципальные ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 14991) starting: 'Глава Ставропольского края встретился с участницей...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15006) starting: 'В России продолжается развитие науки и технологий ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15015) starting: 'Уже несколько лет российская промышленность демонс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15023) starting: '19 новых национальных проектов начали работать в Р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15030) starting: 'Около 580 млн квадратных метров жилья ввели в Росс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15047) starting: 'Начальник ПАСС СК подробно рассказал об СМС-оповещ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15049) starting: 'Экономить электроэнергию гораздо проще, чем кажетс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15054) starting: 'В Ессентуках подготовили к отправке новую партию г...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15063) starting: 'Правительство продолжает реализацию программы расс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15076) starting: 'Губернатор Ставрополья объявил о завершении голосо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15088) starting: 'Состоялся заключительный этап краевого конкурса «Д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15101) starting: 'В краевой столице у мемориала «Вечная Слава» прошл...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15125) starting: 'На прямой линии губернатор Ставрополья уделил вним...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15128) starting: 'Поддержку ветеранам продолжают оказывать на Ставро...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15136) starting: 'Ставропольцы получат СМС-уведомления в случае введ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15140) starting: 'Дороги стали одной главных тем на прямой линии губ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15166) starting: 'Итоги прямой линии губернатора Ставропольского кра...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15169) starting: 'Одной из тем прямой линии губернатора Владимира Вл...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15180) starting: 'Во время прямой линии губернатора Владимира Владим...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15183) starting: 'Участники программы «Герои Ставрополья» смогут пол...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15189) starting: 'В образовательных организациях края проводится бол...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15202) starting: 'Жидкое комплексное органоминеральное удобрение Бог...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15216) starting: 'Стенды с планом эвакуации установили по проспекту ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15243) starting: '🤩 Иллюминация ночной столицы СКФО 📷...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15304) starting: 'Ставрополь встал на выезде — пробка тянется на кил...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15323) starting: '😍 Апрель в великолепном Кисловодске 📹...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15353) starting: '‼открыт прием на БЕСПЛАТНУЮ КОНСУЛЬТАЦИЮ к ведущем...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15412) starting: 'На Ставрополье задержали неудачливых грабителей ба...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15424) starting: '😍 Архитектура старого Кисловодска Когда-то город б...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15475) starting: 'Более 100 нарушений нашла Госавтоинспекция Ставроп...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15480) starting: 'Рэпер ST выступит на Крепостной горе 4 мая в Ставр...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15487) starting: 'Информацию о попытке взрыва банкомата в Ставрополе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15521) starting: 'Дизайн новой 5-тысячной купюры показала ставрополь...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15537) starting: 'Мужской Монастырь Иконы Пресвятой Богородицы “Всех...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15551) starting: 'Метеозависимым приготовиться: мощная магнитная бур...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15577) starting: '"Немецкий" мост в Мамайском лесу Реконструкция с и...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15595) starting: 'Небо в Юго-Западном районе Ставрополя Что-то намеч...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15607) starting: '3D-копии 12 достопримечательностей Ставрополя появ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15612) starting: 'В Ставрополе завершен ремонт аварийного дома на ул...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15615) starting: 'Краски весеннего Пятигорска 📸 [club211425108|@mila...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15616) starting: 'Новый способ оплаты будут тестировать 4 месяца в о...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15619) starting: '86-летний жених и 77-летняя невеста стали самой во...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15624) starting: 'ФСБ опубликовало кадры задержания 41-летнего терро...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15627) starting: '«Образцовая» ночлежка Ставрополя конца XIX века Но...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15633) starting: 'Автономные троллейбусы будут курсировать в новых р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15638) starting: '☀Ставрополь в лучах апрельского солнца 📸 zarisovki...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15646) starting: '💸Пенсионерка из Ставрополя перевела мошенникам 13 ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15689) starting: 'В этот день горожане по традиции выбивают друг дру...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15783) starting: 'Огни ночного Ставрополя 📷 aesthetics_stav...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15792) starting: 'Построить новые котельные и отремонтировать сети п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15794) starting: 'Губернатор разъяснил, как будут работать школы при...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15817) starting: 'Ребенок на Военном городке в Ставрополе держит в с...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15845) starting: '*** До свидания, друзья, говорит, и до встречи, до...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15881) starting: '❗️На территории Ставропольского края ОТБОЙ БЕСПИЛО...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15892) starting: '⚡️Путин разрешил бахнуть пивка в пятницу «Одну-две...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15945) starting: 'Найдено самое милое в Ставрополе объявление о врем...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15946) starting: 'Почти 16 миллионов сим-карт было заблокировано в Р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15982) starting: 'Взорвать банкомат в Ставрополе попытался неизвестн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 15998) starting: '❗️В трех округах Ставрополья началась тотальная пр...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16009) starting: 'Петербурженка осталась без дорогого телефона, прое...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16058) starting: '⚡️В Ставропольском крае объявлена беспилотная опас...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16065) starting: 'Министр ЖКХ края Александр Рябикин рассказал, поче...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16105) starting: '«Молодая Гвардия Единой России» помогает участника...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16109) starting: 'Ставропольчанка делится мнением: "Очень люблю наш ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16117) starting: '❗️В 11 регионах России, включая Ставрополье, пресе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16128) starting: 'Губернатор поздравил казачью бригаду "Терек" с дат...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16144) starting: '🐥Диких птиц в клетках заметили у собора Андрея Пер...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16184) starting: 'Розовое облако цветущей сакуры в ботаническом саду...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16236) starting: 'продам подсы 3 с кейсом ОРИГИНАЛ ,состояние отличн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16246) starting: 'Продам велосипед фирмы Stels модель navigator 400 ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16257) starting: 'тумба для ванной с раковиной бу 2000₽ комод бу 200...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16291) starting: 'Продам новую детскую кровать Спальное место 190*90...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16309) starting: 'Продаю ваз 2114 2012 г, 265000 р...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16322) starting: 'Продам кровать, новая, массив бука, Спальное место...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16339) starting: 'продаю кальян, в отличном состоянии цена 15к, торг...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16344) starting: 'Продам воздушку Байкал MP 512 4.5mm Фото и все воп...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16345) starting: 'Айфон 11 128 АКБ 77% Один владелец не вскрывался л...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16346) starting: 'Ветровка подростковая мужская, Китай Размер: 46 Це...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16378) starting: 'Продаю айфон Х Акб 91% держит хорошо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16385) starting: 'Продается уютная дача с домом 24 кв.м с земельным ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16393) starting: 'Здравствуйте Айфон 13 синий фото нет в идеале теле...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16396) starting: 'Ас салам уалекум 2014 год ,есть кондер,есть страхо...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16448) starting: 'Ищете качественные инертные материалы по доступным...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16469) starting: '2008 год Мотор Масло не ест не троит Коробка в отл...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16475) starting: 'Продам резину 3000...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16476) starting: 'https://vk.com/avtostav126 Продаются динамики 20 с...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16477) starting: 'Продаётся кальян: Цена: 3000р Технически в идеальн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16483) starting: 'Стульчик-качели 2в1 babyton в хорошем состоянии 50...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16492) starting: 'Mercedes E200 , 2010 , 190т В идеале 💰1.500 торг 🌆...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16510) starting: 'Новые,не пользовались Маршал 1200 Джбль 600 Редми ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16511) starting: 'Продам ,в отличном состоянии,уши 5к Мышь 4к...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16512) starting: 'Продам игровые автоматы, по всем вопросам в лс Цен...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16534) starting: 'продам за 3к поду полтора месяца причина продажи -...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16535) starting: '64 gb Внешнее состояние 5/5✅...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16553) starting: '⛔️☦Пасхальные ограничения: какие улицы перекроют в...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16610) starting: '⚡️Задержанный в Пятигорске террорист планировал по...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16619) starting: '⛔️На юге и в центре Ставрополя временно перекроют ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16657) starting: '⚡️❗️Ссора с братом, а не нападение: в Кисловодске ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16665) starting: 'Курортный город с военной историей: Железноводск ж...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16678) starting: '❗️Жительница Ставрополя перевела мошенникам более ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16684) starting: 'erid: 2VtzqvVMHGe Весна в самом разгаре, а муж так...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16698) starting: '⚡В Кисловодске расследуют нападение детей на 8-лет...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16707) starting: '✨Названы популярные артисты, которые выступят на ф...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16716) starting: '⚡️Временное отключение электричества в 45 населенн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16717) starting: '❗ФСБ задержала уроженца Центральной Азии, готовивш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16723) starting: '😱Гибель мужчины в лопастях комбайна скрыли в Предг...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16736) starting: '💸Миллионы рублей, тысячи евро и сотни долларов кон...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16744) starting: 'Как с помощью реальных технологий увеличить поток ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16745) starting: '👶В регионе открылись первые пункты проката вещей д...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16751) starting: 'Ставрополец стал лауреатом национальной премии «Се...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16761) starting: 'erid: 2VtzqwFVkSN 🛠...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16805) starting: 'Авария на водоводе: восемь населенных пунктов Став...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16813) starting: '🏠 В Ставрополе пока не будут отключать отопление В...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16829) starting: '✍Глава Ставрополья отчитался о проектах водоснабже...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16836) starting: '⚡️Участник наркогруппы получил 15 лет колонии за п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16845) starting: 'Команда [https://vk.com/vepri26|«Вепрь»] из Михайл...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16869) starting: '🧺Стоимость минимальной продуктовой корзины на Став...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16914) starting: '❗️😡В Ставрополе заключен под стражу подозреваемый ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16921) starting: '🕊В школе Ставрополя посмертно награждённому Герою ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16928) starting: '⛲️В Ставрополе фонтаны с песнями военных лет зараб...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16940) starting: '😵‍💫Солнечные вспышки класса M вызовут магнитные бу...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16948) starting: '⚡️В подвале многоквартирного дома обнаружили похож...'
Warning: No matching JSON source for #newpar sentence (sent_id: 16954) starting: '🍒Из-за заморозков урожай косточковых деревьев на С...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17056) starting: '🎖Судьбу ставропольского лётчика, пропавшего без ве...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17069) starting: '🚌За высадку детей-безбилетников перевозчикам грози...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17074) starting: '🔑Месть квартиросъёмщика: за ложный донос мужчина п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17079) starting: '❗️Следователи восстановили картину убийства 20-лет...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17092) starting: '📈Численность населения Ставрополья превысила 2,8 м...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17099) starting: '❗️Госавтоинспекция бьет тревогу: подростки гибнут ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17110) starting: '⚡️Водоканал оштрафован на 100 тысяч рублей за нару...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17114) starting: '😱Водитель со 117 штрафами сбил ребёнка на «зебре» ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17121) starting: '50 млн рублей вложил ставропольский бизнес в разви...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17130) starting: '⚡️❗️Движение в сторону Грузии для всех видов транс...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17136) starting: '⚡️Ночная авария с несовершеннолетним водителем при...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17144) starting: '⚡Российские ПВО сбили 36 украинских беспилотников ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17151) starting: '❗На территории Ставропольского края ОТБОЙ БЕСПИЛОТ...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17154) starting: 'В ночь на 12 апреля в Ставропольском крае объявлен...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17161) starting: '⚡️Партия «Новые люди» заявила об участии в выборах...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17169) starting: '🤦‍♀«Фаталити»: пьяная драка в приёмном покое оберн...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17181) starting: '⚡️Семеро участников банды, торговавшей людьми, пол...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17199) starting: '🔥Неосторожность при курении могла стать причиной п...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17219) starting: 'В Ставрополе представили наставников проекта Бизне...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17226) starting: '😱Трагедия на перекрёстке: велосипедист погиб под к...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17235) starting: '❗️Полиция задержала двух подозреваемых в подрыве б...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17304) starting: '⚡️Сотню нарушений выявила тотальная проверка автот...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17311) starting: '💍Более 600 пар свяжут себя узами брака после Пасхи...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17316) starting: '❗️Из пожара в доме на улице Бруснёва спасли 9 чело...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17320) starting: '🔥Два человека пострадали в пожаре на северо-западе...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17329) starting: 'Николай Великдань высказался о роли советского нар...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17344) starting: '😱Автомобиль наехал на бордюр и опрокинулся на прос...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17354) starting: '❗️Неизвестный взорвал банкомат в Ставрополе Происш...'
Warning: No matching JSON source for #newpar sentence (sent_id: 17363) starting: 'В сетевые магазины Фикс-Прайс в г.Ставрополь, с.Ви...'

'''

In [ ]:
import re

sent_ids = [int(m) for m in re.findall(r"sent_id:\s*(\d+)", failed_ids)]

print(sent_ids)

[45, 74, 92, 156, 278, 310, 352, 409, 412, 423, 545, 558, 564, 575, 668, 705, 718, 863, 900, 915, 1046, 1093, 1127, 1153, 1283, 1407, 1427, 1440, 1463, 1581, 1620, 1645, 1740, 1860, 1910, 1977, 2177, 2204, 2220, 2222, 2249, 2276, 2285, 2296, 2309, 2316, 2319, 2339, 2350, 2357, 2411, 2418, 2566, 2611, 2669, 2731, 2855, 2951, 3009, 3010, 3195, 3202, 3212, 3227, 3268, 3269, 3274, 3307, 3369, 3423, 3464, 3607, 3634, 3683, 3766, 3811, 3886, 3954, 3979, 3986, 4004, 4015, 4016, 4135, 4215, 4263, 4269, 4283, 4333, 4425, 4492, 4517, 4563, 4596, 4614, 4632, 4663, 4717, 4718, 4760, 4767, 4836, 4846, 4861, 4868, 4880, 4906, 4914, 5011, 5074, 5088, 5093, 5094, 5148, 5166, 5170, 5189, 5196, 5211, 5254, 5271, 5304, 5305, 5375, 5428, 5434, 5448, 5452, 5453, 5600, 5664, 5706, 5751, 5796, 5800, 5838, 5888, 5897, 5907, 5908, 5944, 5964, 5976, 6111, 6131, 6143, 6182, 6214, 6242, 6318, 6319, 6369, 6399, 6466, 6556, 6562, 6565, 6640, 6666, 6743, 6780, 6800, 6808, 6833, 6855, 6858, 6898, 6938, 7049, 7084, 72

In [ ]:
len(sent_ids)

580